In [1]:
##
import os 
from langchain_groq import ChatGroq
from dotenv import load_dotenv

load_dotenv()

GROQ_API_KEY = os.getenv("GROQ_API_KEY")



llm = ChatGroq(model = "llama-3.2-1b-preview",
            groq_api_key = GROQ_API_KEY)

In [11]:
import spacy

# Load the spaCy English model
nlp = spacy.load("en_core_web_sm")

# A simple mapping from common natural language phrases to OData query components
NL_TO_OLQUERY_MAP = {
    "find": "select",
    "filter": "filter",
    "where": "filter",
    "order by": "orderby",
    "greater than": "gt",
    "less than": "lt",
    "equals": "eq",
    "not equal": "ne"
}

def parse_natural_language(query):
    """
    Parse the natural language query to extract key information.
    """
    doc = nlp(query.lower())
    
    # Placeholder for extracted parts of the OData query
    select_part = []
    filter_part = []
    orderby_part = None
    
    # Parse each token and map to OData equivalent
    for token in doc:
        text = token.text
        
        if text in NL_TO_OLQUERY_MAP:
            odata_term = NL_TO_OLQUERY_MAP[text]
            
            if odata_term == "select":
                select_part.append(token.head.text)
            elif odata_term == "filter":
                filter_part.append(token.head.text)
            elif odata_term in ('gt', 'lt', 'eq', 'ne'):
                filter_part.append(f"{token.head.text} {odata_term} {token.nbor().text}")
            elif odata_term == "orderby":
                orderby_part = token.head.text
        elif token.dep_ == "ROOT":
            # Try to interpret the main intent (like finding/filtering)
            if token.lemma_ == "find":
                select_part.append(token.head.text)
    
    # Construct OData query
    odata_query = construct_odata_query(select_part, filter_part, orderby_part)
    return odata_query


def construct_odata_query(select_part, filter_part, orderby_part):
    """
    Construct the OData query from parsed components.
    """
    odata_query = "$select=" + ",".join(select_part)
    
    if filter_part:
        odata_query += "&$filter=" + " and ".join(filter_part)
        
    if orderby_part:
        odata_query += "&$orderby=" + orderby_part
    
    return odata_query


# Sample usage
natural_language_query = "Find customers where age greater than 30."
odata_query = parse_natural_language(natural_language_query)
print("OData Query:", odata_query)


OData Query: $select=find&$filter=age


In [8]:
from langchain.tools import Tool
import spacy

# Load the spaCy English model
nlp = spacy.load("en_core_web_sm")

# Natural Language to OData Mapping
NL_TO_OLQUERY_MAP = {
    "find": "select",
    "filter": "filter",
    "where": "filter",
    "order by": "orderby",
    "greater than": "gt",
    "less than": "lt",
    "equals": "eq",
    "not equal": "ne"
}

def parse_natural_language(query: str) -> str:
    """
    Parse the natural language query to extract key information.
    """
    doc = nlp(query.lower())
    
    # Placeholder for extracted parts of the OData query
    select_part = []
    filter_part = []
    orderby_part = None
    
    # Parse each token and map to OData equivalent
    for token in doc:
        text = token.text
        
        if text in NL_TO_OLQUERY_MAP:
            odata_term = NL_TO_OLQUERY_MAP[text]
            
            if odata_term == "select":
                select_part.append(token.head.text)
            elif odata_term == "filter":
                filter_part.append(token.head.text)
            elif odata_term == "orderby":
                orderby_part = token.head.text
        elif token.dep_ == "ROOT":
            # Try to interpret the main intent (like finding/filtering)
            if token.lemma_ == "find":
                select_part.append(token.head.text)
    
    # Construct OData query
    return construct_odata_query(select_part, filter_part, orderby_part)

def construct_odata_query(select_part, filter_part, orderby_part):
    """
    Construct the OData query from parsed components.
    """
    odata_query = "$select=" + ",".join(select_part)
    
    if filter_part:
        odata_query += "&$filter=" + " and ".join(filter_part)
        
    if orderby_part:
        odata_query += "&$orderby=" + orderby_part
    
    return odata_query


# Define the LangChain Tool
def nl_to_odata_tool(query: str) -> str:
    """
    A LangChain tool that converts natural language queries into OData queries.
    """
    odata_query = parse_natural_language(query)
    return odata_query


# Define the LangChain Tool object
nl_to_odata_langchain_tool = Tool(
    name="Natural Language to OData Converter",
    description="Converts natural language queries into OData query syntax.",
    func=nl_to_odata_tool
)

# Example Usage in a LangChain Agent (if needed)
# from langchain.agents import initialize_agent, Tool, AgentType
# from langchain.llms import OpenAI
# 
# llm = OpenAI(temperature=0)  # Or your preferred LLM
# tools = [nl_to_odata_langchain_tool]
# 
# agent = initialize_agent(tools, llm, agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION)
# agent.run("Find customers where age is greater than 30 and order by name")


In [9]:
d_1 = nl_to_odata_tool("Find customers where age greater than 30.")
d_1

'$select=find&$filter=age'

In [ ]:
from langchain.tools import Tool

from textblob import TextBlob

# Natural Language to OData Mapping
NL_TO_OLQUERY_MAP = {
    "find": "select",
    "filter": "filter",
    "where": "filter",
    "order by": "orderby",
    "greater than": "gt",
    "less than": "lt",
    "equals": "eq",
    "not equal": "ne"
}

def parse_natural_language(query: str) -> str:
    """
    Parse the natural language query to extract key information.
    """
    blob = TextBlob(query.lower())
    
    # Placeholder for extracted parts of the OData query
    select_part = []
    filter_part = []
    orderby_part = None
    
    # Parse each word and map to OData equivalent
    words = blob.words
    for i, word in enumerate(words):
        if word in NL_TO_OLQUERY_MAP:
            odata_term = NL_TO_OLQUERY_MAP[word]
            
            if odata_term == "select" and i < len(words) - 1:
                select_part.append(words[i + 1])
            elif odata_term == "filter" and i < len(words) - 1:
                filter_part.append(f"{words[i + 1]} {NL_TO_OLQUERY_MAP.get(words[i + 2], '')} {words[i + 3]}")
            elif odata_term == "orderby" and i < len(words) - 1:
                orderby_part = words[i + 1]
    
    # Construct OData query
    return construct_odata_query(select_part, filter_part, orderby_part)

def construct_odata_query(select_part, filter_part, orderby_part):
    """
    Construct the OData query from parsed components.
    """
    odata_query = "$select=" + ",".join(select_part)
    
    if filter_part:
        odata_query += "&$filter=" + " and ".join(filter_part)
        
    if orderby_part:
        odata_query += "&$orderby=" + orderby_part
    
    return odata_query


# Define the LangChain Tool
def nl_to_odata_tool(query: str) -> str:
    """
    A LangChain tool that converts natural language queries into OData queries.
    """
    odata_query = parse_natural_language(query)
    return odata_query


# Define the LangChain Tool object
nl_to_odata_langchain_tool = Tool(
    name="Natural Language to OData Converter",
    description="Converts natural language queries into OData query syntax.",
    func=nl_to_odata_tool
)

# Example Usage in a LangChain Agent (if needed)
# from langchain.agents import initialize_agent, Tool, AgentType
# from langchain.llms import OpenAI
# 
# llm = OpenAI(temperature=0)  # Or your preferred LLM
# tools = [nl_to_odata_langchain_tool]
# 
# agent = initialize_agent(tools, llm, agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION)
# agent.run("Find customers where age is greater than 30 and order by name")


In [ ]:
import spacy

# Load the spaCy English model
nlp = spacy.load("en_core_web_sm")

# A simple mapping from common natural language phrases to OData query components
NL_TO_OLQUERY_MAP = {
    "find": "select",
    "filter": "filter",
    "where": "filter",
    "order by": "orderby",
    "greater than": "gt",
    "less than": "lt",
    "equals": "eq",
    "not equal": "ne"
}



def parse_natural_language(query):
    doc = nlp(query.lower())
    
    select_part = []
    filter_part = []
    orderby_part = None
    
    for token in doc:
        text = token.text
        
        if text in NL_TO_OLQUERY_MAP:
            odata_term = NL_TO_OLQUERY_MAP[text]
            
            if odata_term == "select":
                select_part.append(token.head.text)
            elif odata_term == "filter":
                filter_part.append(token.head.text)
            elif odata_term in ("gt", "lt", "eq", "ne"):
                filter_part.append(f"{token.head.text} {odata_term} {token.nbor().text}")
            elif odata_term == "orderby":
                orderby_part = token.head.text
        elif token.dep_ == "ROOT":
            if token.lemma_ == "find":
                select_part.append(token.head.text)
    
    odata_query = construct_odata_query(select_part, filter_part, orderby_part)
    return odata_query


### Running

In [12]:
import spacy

# Load the spaCy English model
nlp = spacy.load("en_core_web_sm")

# A mapping from common natural language phrases to OData query components
NL_TO_OLQUERY_MAP = {
    "find": "select",
    "get": "select",
    "filter": "filter",
    "where": "filter",
    "order by": "orderby",
    "greater than": "gt",
    "less than": "lt",
    "equals": "eq",
    "not equal": "ne",
    "equal to": "eq"
}

def parse_natural_language(query):
    """
    Parse the natural language query to extract key information.
    """
    doc = nlp(query.lower())
    
    # Placeholder for extracted parts of the OData query
    select_part = []
    filter_part = []
    orderby_part = None
    
    # Parse each token and map to OData equivalent
    for i, token in enumerate(doc):
        if token.text == "find" or token.text == "get":
            # The next token is likely what we're selecting
            if i + 1 < len(doc):
                select_part.append(doc[i+1].text)
        elif token.text == "where":
            # Look for the condition after "where"
            if i + 3 < len(doc):
                field = doc[i+1].text
                condition = " ".join([t.text for t in doc[i+2:i+4]])
                if condition in NL_TO_OLQUERY_MAP:
                    odata_condition = NL_TO_OLQUERY_MAP[condition]
                    if i + 4 < len(doc):
                        value = doc[i+4].text
                        filter_part.append(f"{field} {odata_condition} {value}")
    
    # Construct OData query
    odata_query = construct_odata_query(select_part, filter_part, orderby_part)
    return odata_query

def construct_odata_query(select_part, filter_part, orderby_part):
    """
    Construct the OData query from parsed components.
    """
    odata_query = "$select=" + ",".join(select_part) if select_part else ""
    
    if filter_part:
        odata_query += "&$filter=" + " and ".join(filter_part)
        
    if orderby_part:
        odata_query += "&$orderby=" + orderby_part
    
    return odata_query

# Sample usage
natural_language_query = "Find customers where age greater than 30"
odata_query = parse_natural_language(natural_language_query)
print("Natural Language Query:", natural_language_query)
print("OData Query:", odata_query)

Natural Language Query: Find customers where age greater than 30
OData Query: $select=customers&$filter=age gt 30
